In [8]:
import requests
import pandas as pd
import hopsworks

conn = hopsworks.connection()
HEILBONN_API_KEY = conn.get_secrets_api().get_secret("HEILBONN_API_KEY").value
    
# Define the API URL and API key
url = "https://34.149.159.161.nip.io/bildungscampus/iotplatform/mensaoccupancy/v1/authGroup/mensa_occupancy_devices/entityId?page=0"

# Set up headers with API key
headers = {
    'x-apikey': HEILBONN_API_KEY
}

# Send GET request to the API
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    
    # Initialize lists to store data
    location_names = []
    location_ts = []
    out_counts = []
    out_count_ts = []
    occupancies = []
    occupancy_ts = []
    occupancy_relatives = []
    occupancy_relative_ts = []
    in_counts = []
    in_count_ts = []
    capacities = []
    capacity_ts = []

    # Extract the necessary fields from the JSON response
    for entity in data['entities']:
        location_names.append(entity['SERVER_ATTRIBUTE']['locationName']['value'])
        location_ts.append(entity['SERVER_ATTRIBUTE']['locationName']['ts'])
        
        out_counts.append(entity['TIME_SERIES']['outCount']['value'])
        out_count_ts.append(entity['TIME_SERIES']['outCount']['ts'])
        
        occupancies.append(entity['TIME_SERIES']['occupancy']['value'])
        occupancy_ts.append(entity['TIME_SERIES']['occupancy']['ts'])
        
        occupancy_relatives.append(entity['TIME_SERIES']['occupancyRelative']['value'])
        occupancy_relative_ts.append(entity['TIME_SERIES']['occupancyRelative']['ts'])
        
        in_counts.append(entity['TIME_SERIES']['inCount']['value'])
        in_count_ts.append(entity['TIME_SERIES']['inCount']['ts'])
        
        capacities.append(entity['TIME_SERIES']['capacity']['value'])
        capacity_ts.append(entity['TIME_SERIES']['capacity']['ts'])

    # Create a DataFrame from the lists
    df = pd.DataFrame({
        'location_name': location_names,
        'location_ts': location_ts,
        'out_count': out_counts,
        'out_count_ts': out_count_ts,
        'occupancy': occupancies,
        'occupancy_ts': occupancy_ts,
        'occupancy_relative': occupancy_relatives,
        'occupancy_relative_ts': occupancy_relative_ts,
        'in_count': in_counts,
        'in_count_ts': in_count_ts,
        'capacity': capacities,
        'capacity_ts': capacity_ts
    })

    # Convert UNIX timestamps to pandas datetime
    df['location_ts'] = pd.to_datetime(df['location_ts'], unit='ms')
    df['out_count_ts'] = pd.to_datetime(df['out_count_ts'], unit='ms')
    df['occupancy_ts'] = pd.to_datetime(df['occupancy_ts'], unit='ms')
    df['occupancy_relative_ts'] = pd.to_datetime(df['occupancy_relative_ts'], unit='ms')
    df['in_count_ts'] = pd.to_datetime(df['in_count_ts'], unit='ms')
    df['capacity_ts'] = pd.to_datetime(df['capacity_ts'], unit='ms')

    print(df)
else:
    print(f"Failed to retrieve data: {response.status_code} - {response.text}")


Connected. Call `.close()` to terminate connection gracefully.
      location_name             location_ts out_count            out_count_ts  \
0  Mensa Gebäude U8 2024-08-16 11:25:39.300       242 2024-08-16 11:25:38.791   

  occupancy            occupancy_ts occupancy_relative  \
0        51 2024-08-16 11:25:38.791              0.102   

    occupancy_relative_ts in_count             in_count_ts capacity  \
0 2024-08-16 11:25:38.791      293 2024-08-16 11:25:38.791      500   

              capacity_ts  
0 2024-08-16 11:25:38.791  


In [6]:
import hopsworks
fs = hopsworks.login().get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/16503
Connected. Call `.close()` to terminate connection gracefully.


In [7]:
fg = fs.get_or_create_feature_group(name="mensa_live",
                                    version=1, 
                                    primary_key=['location_name', 'location_ts'], 
                                    event_time="occupancy_ts",
                                    online_enabled=True)

In [8]:
fg.insert(df)

Feature Group created successfully, explore it at 
https://snurran.hops.works/p/16503/fs/16451/fg/17430


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: mensa_live_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/16503/jobs/named/mensa_live_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fba4fd43ac0>, None)